In [87]:
import itertools as it
import numpy as np

def kendallTau(A, B=None):
    # if any partial is B
    if B is None : B = list(range(len(A)))
    n = len(A)
    pairs = it.combinations(range(n), 2)
    distance = 0
    for x, y in pairs:
        #if not A[x]!=A[x] and not A[y]!=A[y]:#OJO no se check B
        a = A[x] - A[y]
        try:
            b = B[x] - B[y]# if discordant (different signs)
        except:
            print("ERROR kendallTau, check b",A, B, x, y)
        # print(b,a,b,A, B, x, y,a * b < 0)
        if (a * b < 0):
            distance += 1
    return distance
    
## number of perms at each dist
def num_perms_at_dist(n):
    sk = np.zeros((n+1,int(n*(n-1)/2+1)))
    for i in range(n+1):
        sk[i,0] = 1
    for i in range(1,1+n):
        for j in range(1,int(i*(i-1)/2+1)):
            if j - i >= 0 :
                sk[i,j] = sk[i,j-1]+ sk[i-1,j] - sk[i-1,j-i]
            else:
                sk[i,j] = sk[i,j-1]+ sk[i-1,j]
    return sk.astype(np.uint64)

def v2ranking(v, n): ##len(v)==n, last item must be 0
    # n = len(v)
    rem = list(range(n))
    rank = np.array([np.nan]*n)# np.zeros(n,dtype=np.int)
    # print(v,rem,rank)
    for i in range(len(v)):
        rank[i] = rem[v[i]]
        rem.pop(v[i])
    return rank.astype(int)#[i+1 for i in permut];



## random permutations at distance
def random_perm_at_dist(n, dist, sk):
    # param sk is the results of the function num_perms_at_dist(n)
    i = 0
    probs = np.zeros(n+1)
    v = np.zeros(n,dtype=int)
    while i<n and dist > 0 :
        rest_max_dist = (n - i - 1 ) * ( n - i - 2 ) / 2
        if rest_max_dist  >= dist:
            probs[0] = sk[n-i-1,dist]
        else:
            probs[0] = 0
        mi = min(dist + 1 , n - i )
        for j in range(1,mi):
            if rest_max_dist + j >= dist: probs[j] = sk[n-i-1, dist-j]
            else: probs[ j ] = 0
        v[i] = np.random.choice(mi,1,p=probs[:mi]/probs[:mi].sum())
        dist -= v[i]
        i += 1
    return v2ranking(v,n)











# end


we have a permutation $s$ and we want a permutation $X$ at distance $d$ from $s$, i.e., $d(s,X)= d$

In [88]:
n = 8
d = 4
sk = num_perms_at_dist(n) #auxiliar function
s = np.random.permutation(n) 

#first, we generate a permutation pi, such that d(pi,e)=d
pi = random_perm_at_dist(n, d, sk)
composition = pi[s] #now we compose it with s, because
#d(pi,e)=d => d(pi[s],e[s])=d =>  d(pi[s],s)=d =>   X = pi[s] 
kendallTau(pi), kendallTau(s, composition)

(4, 4)